In [5]:
import tensorflow as tf
print(f"Tensorflow version : {tf.__version__}")

Tensorflow version : 2.2.0-rc1


In [6]:
from tensorflow.python.ops import sparse_ops
import tensorflow.keras.backend as K
import pandas as pd
# import matplotlib.pyplot as plt
import os
assert tf.__version__=='2.2.0-rc1'
# assert tf.test.is_gpu_available()
K.clear_session()
%load_ext tensorboard
import matplotlib.pyplot as plt
import numpy as np

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [7]:
from ipynb.fs.full.clustering import cluster_maker, cluster_splitter, scalar_features, tf_serialize_example

## Parser

In [34]:
feature_description = {
    'eventId': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
    'image': tf.io.FixedLenFeature([3], tf.string, default_value=["",]*3)
}

feature_shape = {
    'eventId': tf.TensorShape([1,]),
    'image': tf.TensorShape([568, 568, 2])
}

def parser_fn(proto):
    serialized = tf.io.parse_single_example(proto, feature_description)
    deserialized = {k: (tf.sparse.to_dense(sparse_ops.deserialize_sparse(v, K.floatx()))
                        if k != 'eventId' else v) for k,v in serialized.items()}
    [deserialized[k].set_shape(feature_shape[k]) for k in deserialized.keys()]
    x = deserialized['image']
    x = tf.expand_dims(x, axis=0)
    x = K.pool2d(x, pool_size=(2, 2), strides=(2, 2), pool_mode='avg')
    # sum instead of avg
#     x = 4.*x
    deserialized['image'] = tf.squeeze(x)
    deserialized['S_image'] = deserialized['image'][:,:,0]
    deserialized['C_image'] = deserialized['image'][:,:,1]
    
    # set EM633
    deserialized['tseed'] = tf.constant(6., tf.float32)
    deserialized['tneighbour'] = tf.constant(3., tf.float32)
    deserialized['tcell'] = tf.constant(3., tf.float32)
    deserialized['tenergy'] = tf.constant(1000., tf.float32)
    deserialized['tlocmax'] = tf.constant(500., tf.float32)
    deserialized['tnum'] = tf.constant(3, tf.int32)
    
    return deserialized

In [39]:
DATA_DIR = ''
BATCH_SIZE = 16
BUFFER_SIZE = 10

filename = [os.path.join(DATA_DIR, 'B4.tfrecord')]
dataset = tf.data.TFRecordDataset(filename, compression_type='GZIP', buffer_size=BUFFER_SIZE)
dataset = dataset.map(parser_fn)
print(dataset)
clusters_dataset = dataset.take(10).map(lambda x: cluster_maker(x, im='S_image'))
clusters_dataset = clusters_dataset.map(lambda x: cluster_splitter(x, im='S_image'))
features_dataset = clusters_dataset.map(scalar_features)
serialized_features_dataset = features_dataset.map(tf_serialize_example)
print(features_dataset)
# print(clusters_dataset)

<MapDataset shapes: {eventId: (1,), image: (284, 284, 2), S_image: (284, 284), C_image: (284, 284), tseed: (), tneighbour: (), tcell: (), tenergy: (), tlocmax: (), tnum: ()}, types: {eventId: tf.int64, image: tf.float32, S_image: tf.float32, C_image: tf.float32, tseed: tf.float32, tneighbour: tf.float32, tcell: tf.float32, tenergy: tf.float32, tlocmax: tf.float32, tnum: tf.int32}>
<MapDataset shapes: {eventId: (1,), image: (284, 284, 2), S_image: (284, 284), C_image: (284, 284), tseed: (), tneighbour: (), tcell: (), tenergy: (), tlocmax: (), tnum: (), proto: (None, 3), cluster: (None, 4), feature: (None, 5)}, types: {eventId: tf.int64, image: tf.float32, S_image: tf.float32, C_image: tf.float32, tseed: tf.float32, tneighbour: tf.float32, tcell: tf.float32, tenergy: tf.float32, tlocmax: tf.float32, tnum: tf.int32, proto: tf.int64, cluster: tf.float32, feature: tf.float32}>


In [40]:
filename = 'test123.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

locmax [[19 126 150]]
seedlist [[20 126 150]]
locmax []
seedlist []
locmax []
seedlist []
locmax []
seedlist []
locmax []
seedlist []
locmax []
seedlist []
locmax []
seedlist []
locmax []
seedlist []
locmax []
seedlist []
locmax []
seedlist []


In [41]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
for raw_record in raw_dataset.take(2):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "S_hot"
    value {
      float_list {
        value: 0.11785701662302017
      }
    }
  }
  feature {
    key: "S_rad_mean"
    value {
      float_list {
        value: 2.3386428356170654
      }
    }
  }
  feature {
    key: "S_sum"
    value {
      float_list {
        value: 6445.4306640625
      }
    }
  }
  feature {
    key: "cluster_comi"
    value {
      float_list {
        value: 126.17872619628906
      }
    }
  }
  feature {
    key: "cluster_comj"
    value {
      float_list {
        value: 150.4558563232422
      }
    }
  }
  feature {
    key: "eventId"
    value {
      int64_list {
        value: 0
      }
    }
  }
}

features {
  feature {
    key: "S_hot"
    value {
      float_list {
      }
    }
  }
  feature {
    key: "S_rad_mean"
    value {
      float_list {
      }
    }
  }
  feature {
    key: "S_sum"
    value {
      float_list {
      }
    }
  }
  feature {
    key: "cluster_comi"
    value {
      float_lis

In [50]:
for output in clusters_dataset.take(4):
    output

locmax [[19 126 150]]
seedlist [[20 126 150]]
locmax []
seedlist []
locmax []
seedlist []
locmax []
seedlist []


In [51]:
import ROOT
from array import array
C = output['cluster'].numpy()

h = ROOT.TH2F("","", 284, 0., 284., 284, 0., 284.)
c = ROOT.TCanvas("c1","c1", 800, 400)
c.Divide(2)
ROOT.gStyle.SetOptStat(0)
palette = [ROOT.kRed, ROOT.kGreen, ROOT.kYellow, ROOT.kBlue]
for l in range(2):
    c.cd(l+1)
    ROOT.gPad.SetLeftMargin(0.05)
    ROOT.gPad.SetBottomMargin(0.05)
    ROOT.gPad.SetRightMargin(0.15)
    ROOT.gPad.SetTopMargin(0.15)
#     Cl = C[C[...,3]==l]
    Cl = C
#     idx = np.array([1,2,4])
    idx = np.array([1,2,3])
    hk = []
    u = np.unique(Cl[:,0], axis=0).astype(int)
    htot = h.Clone()
    arr = array('d',[0.5])
    for k in range(len(u)):
        for i,j,x in Cl[Cl[...,0]==u[k]][:,idx]:
#         for i,j,x in Cl[:,idx]:

            hk.append(h.Clone())
            hk[k].Fill(j,i,x)
            htot.Add(hk[k])
    htot.DrawCopy("COLZ")

    for k in range(len(u)):
        hk[k].SetLineColor(palette[k])
        hk[k].SetLineWidth(1)
        hk[k].SetContour(1, arr)
        hk[k].DrawCopy("cont3 list same")

c.Draw()

ValueError: cannot reshape array of size 0 into shape (0,newaxis)

Warning in <TCanvas::Constructor>: Deleting canvas with same name: c1
